<a href="https://colab.research.google.com/github/21092004Goda/data_anal/blob/main/RAG_system_lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai

In [ ]:
import arxiv
import pandas as pd

def fetch_arxiv_ml_articles(max_results=10):

    client = arxiv.Client()

    search = arxiv.Search(
        query="cat:cs.LG",
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    articles_data = []

    for result in client.results(search):
        article_info = {
            'title': result.title,
            'authors': [author.name for author in result.authors],
            'abstract': result.summary,
            'pdf_url': result.pdf_url,
            'published': result.published.date()
        }
        articles_data.append(article_info)

    df = pd.DataFrame(articles_data)
    return df

if __name__ == "__main__":
    df_articles = fetch_arxiv_ml_articles(max_results=15)

    print(df_articles)

    df_articles.to_csv('arxiv_ml_articles.csv', index=False, encoding='utf-8')
    print("\nДанные сохранены в файл 'arxiv_ml_articles.csv'")

                                                title  \
0   TraceGen: World Modeling in 3D Trace Space Ena...   
1   ToolOrchestra: Elevating Intelligence via Effi...   
2   Matrix: Peer-to-Peer Multi-Agent Synthetic Dat...   
3   Agentic Learner with Grow-and-Refine Multimoda...   
4   On Evolution-Based Models for Experimentation ...   
5   DSD: A Distributed Speculative Decoding Soluti...   
6   Through the telecom lens: Are all training sam...   
7   Escaping the Verifier: Learning to Reason via ...   
8               EvilGenie: A Reward Hacking Benchmark   
9   Continual Error Correction on Low-Resource Dev...   
10  Aligning LLMs Toward Multi-Turn Conversational...   
11  Mechanisms of Non-Monotonic Scaling in Vision ...   
12  Scale-Agnostic Kolmogorov-Arnold Geometry in N...   
13        On the Origin of Algorithmic Progress in AI   
14  Beyond URLs: Metadata Diversity and Position f...   

                                              authors  \
0   [Seungjae Lee, Yoonkyo Jun

In [ ]:
import pandas as pd
import re
from typing import List, Dict

# Загружаем данные
df_articles = pd.read_csv('arxiv_ml_articles.csv')

# Анализируем длину аннотаций
def analyze_abstracts_length(df):
    word_counts = []
    for abstract in df['abstract']:
        words = re.findall(r'\b\w+\b', abstract)
        word_counts.append(len(words))

    df['word_count'] = word_counts
    return word_counts

word_counts = analyze_abstracts_length(df_articles)

print("Статистика по длине аннотаций (в словах):")
print(f"Минимум: {min(word_counts)}")
print(f"Максимум: {max(word_counts)}")
print(f"Среднее: {sum(word_counts)/len(word_counts):.1f}")
print(f"Медиана: {sorted(word_counts)[len(word_counts)//2]}")
print("\nРаспределение длин:")
for count in word_counts:
    print(f"- {count} слов")

Статистика по длине аннотаций (в словах):
Минимум: 131
Максимум: 250
Среднее: 188.5
Медиана: 185

Распределение длин:
- 224 слов
- 217 слов
- 201 слов
- 221 слов
- 250 слов
- 136 слов
- 166 слов
- 177 слов
- 171 слов
- 185 слов
- 201 слов
- 182 слов
- 131 слов
- 201 слов
- 165 слов


In [ ]:
def split_into_chunks(text: str, chunk_size: int = 150, overlap: int = 15) -> List[Dict]:
    """
    Разбивает текст на чанки с перекрытием.

    Args:
        text (str): Исходный текст
        chunk_size (int): Размер чанка в словах
        overlap (int): Перекрытие между чанками в словах

    Returns:
        List[Dict]: Список чанков с метаданными
    """
    # Разбиваем текст на слова
    words = re.findall(r'\b\w+\b', text)

    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk_text = ' '.join(chunk_words)

        chunk_info = {
            'text': chunk_text,
            'start_word': start,
            'end_word': min(end, len(words)),
            'total_words': len(chunk_words),
            'is_complete': end >= len(words)
        }
        chunks.append(chunk_info)

        # Если дошли до конца текста
        if end >= len(words):
            break

        # Перемещаем стартовую позицию с учетом перекрытия
        start += chunk_size - overlap

    return chunks

def create_chunked_dataset(df, chunk_size=150, overlap=15):
    """
    Создает датасет с чанками для всех аннотаций.

    Args:
        df: DataFrame со статьями
        chunk_size: Размер чанка в словах
        overlap: Перекрытие между чанками в словах

    Returns:
        DataFrame с чанками
    """
    chunked_data = []

    for idx, row in df.iterrows():
        chunks = split_into_chunks(row['abstract'], chunk_size, overlap)

        for chunk_idx, chunk in enumerate(chunks):
            chunk_record = {
                'original_article_id': idx,
                'title': row['title'],
                'chunk_id': chunk_idx,
                'chunk_text': chunk['text'],
                'start_word': chunk['start_word'],
                'end_word': chunk['end_word'],
                'total_words_in_chunk': chunk['total_words'],
                'is_final_chunk': chunk['is_complete'],
                'total_words_in_abstract': len(re.findall(r'\b\w+\b', row['abstract']))
            }
            chunked_data.append(chunk_record)

    return pd.DataFrame(chunked_data)

# Создаем чанки
chunk_size = 150
overlap = 15

df_chunks = create_chunked_dataset(df_articles, chunk_size, overlap)

print(f"Исходное количество статей: {len(df_articles)}")
print(f"Получено чанков: {len(df_chunks)}")
print(f"Среднее чанков на статью: {len(df_chunks)/len(df_articles):.1f}")

# Показываем примеры чанков
print("\nПримеры чанков:")
for i, (_, chunk) in enumerate(df_chunks.head(3).iterrows()):
    print(f"\n--- Чанк {i+1} (статья '{chunk['title'][:50]}...') ---")
    print(f"Слова: {chunk['start_word']}-{chunk['end_word']}")
    print(f"Текст: {chunk['chunk_text'][:200]}...")

# Сохраняем результат
df_chunks.to_csv('arxiv_ml_articles_chunks.csv', index=False, encoding='utf-8')
print(f"\nЧанки сохранены в 'arxiv_ml_articles_chunks.csv'")

Исходное количество статей: 15
Получено чанков: 28
Среднее чанков на статью: 1.9

Примеры чанков:

--- Чанк 1 (статья 'TraceGen: World Modeling in 3D Trace Space Enables...') ---
Слова: 0-150
Текст: Learning new robot tasks on new platforms and in new scenes from only a handful of demonstrations remains challenging While videos of other embodiments humans and different robots are abundant differe...

--- Чанк 2 (статья 'TraceGen: World Modeling in 3D Trace Space Enables...') ---
Слова: 135-224
Текст: triplets Pretraining on this corpus produces a transferable 3D motion prior that adapts efficiently with just five target robot videos TraceGen attains 80 success across four tasks while offering 50 6...

--- Чанк 3 (статья 'ToolOrchestra: Elevating Intelligence via Efficien...') ---
Слова: 0-150
Текст: Large language models are powerful generalists yet solving deep and complex problems such as those of the Humanity s Last Exam HLE remains both conceptually challenging and computationally 

In [ ]:
# Векторизация текста и построение векторного хранилища с FAISS
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# Загружаем чанки
df_chunks = pd.read_csv('arxiv_ml_articles_chunks.csv')

# Инициализируем модель для эмбеддингов
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Создаем эмбеддинги для всех чанков
texts = df_chunks['chunk_text'].tolist()
embeddings = model.encode(texts, normalize_embeddings=True)

# Создаем FAISS индекс
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

# Сохраняем векторное хранилище
faiss.write_index(index, 'faiss_arxiv_ml.index')

print("✅ Векторизация завершена!")
print(f"• Обработано чанков: {len(df_chunks)}")
print(f"• Размерность векторов: {dimension}")
print(f"• Векторное хранилище сохранено: faiss_arxiv_ml.index")

✅ Векторизация завершена!
• Обработано чанков: 28
• Размерность векторов: 384
• Векторное хранилище сохранено: faiss_arxiv_ml.index


In [ ]:
# Упрощенный пример улучшенной функции поиска и ответа
def enhanced_search_and_answer(question, top_k=3):
    """Улучшенный поиск: находит релевантные статьи и дает развернутый ответ."""

    # 1. ВЕКТОРНЫЙ ПОИСК: Используем FAISS для нахождения релевантных чанков
    query_embedding = model.encode([question], normalize_embeddings=True).astype('float32')
    distances, indices = index.search(query_embedding, top_k)

    # 2. СБОР КОНТЕКСТА: Готовим информацию для LLM
    context = "Релевантные научные статьи:\n"
    article_titles = []

    for i, idx in enumerate(indices[0]):
        chunk = df_chunks.iloc[idx]
        context += f"\n[{i+1}] Заголовок: {chunk['title']}\n"
        context += f"   Фрагмент: {chunk['chunk_text']}\n"
        article_titles.append(chunk['title'])

    # 3. ГЕНЕРАЦИЯ ОТВЕТА: Используем LLM для синтеза информации
    prompt = f"""
    Ты — научный ассистент. Используй приведенный ниже контекст, чтобы дать развернутый ответ на вопрос.

    {context}

    Вопрос: {question}

    Инструкции к ответу:
    - Будь точным и основывайся только на предоставленном контексте.
    - Если в контексте нет информации для ответа, так и скажи.
    - Выдели основные идеи и, если возможно, укажи, из каких статей они взяты.
    - Ответ должен быть полезен для исследователя в области машинного обучения.
    """

    response = model.generate_content(prompt)
    return response.text, article_titles  # Возвращаем ответ и список использованных статей

# Пример использования полного пайплайна
question = "Какие последние достижения в области эффективного обслуживания больших языковых моделей?"
detailed_answer, used_articles = enhanced_search_and_answer(question)

print(f"❓ Вопрос: {question}")
print(f"💡 Ответ: {detailed_answer}")
print(f"📚 Использованные статьи: {used_articles}")

AttributeError: 'SentenceTransformer' object has no attribute 'generate_content'